In [ ]:
import os
import io
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests
import pandas as pd

In [ ]:
# ----------- CONFIG -----------
# Put either your local path (use raw string if on Windows with parentheses)
local_path = r"C:\Users\LENOVO\Downloads\Lenna_(test_image).png"
# Or use the URL (uncomment to use)
url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"

output_dir = "svd_lenna_outputs"
resize_to = (512, 512)   # set to None to keep original size
ks = [5, 20, 50, 100]


In [ ]:
# ------------------------------

os.makedirs(output_dir, exist_ok=True)

def load_image(path=None, url=None, to_gray=True, resize=None):
    if path is not None and os.path.exists(path):
        img = Image.open(path)
    elif url is not None:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        img = Image.open(io.BytesIO(resp.content))
    else:
        raise ValueError("Provide a valid local path or URL")

    if to_gray:
        img = img.convert("L")
    if resize is not None:
        img = img.resize(resize, Image.LANCZOS)
    return np.array(img, dtype=np.float64)

def svd_decompose(A):
    # economy/truncated SVD
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    return U, s, Vt

def reconstruct_k(U, s, Vt, k):
    k = max(1, min(k, len(s)))  # safety
    Uk = U[:, :k]
    Sk = np.diag(s[:k])
    Vtk = Vt[:k, :]
    A_k = Uk @ Sk @ Vtk
    return A_k

def mse(A, B):
    return float(np.mean((A - B) ** 2))

def psnr(A, B, max_val=255.0):
    m = mse(A, B)
    if m == 0:
        return float("inf")
    return 10.0 * math.log10((max_val ** 2) / m)

def compression_ratio(m, n, k):
    # numbers required = k*(m + n + 1)
    return (k * (m + n + 1)) / (m * n)

def save_image(array, path):
    img = np.clip(array, 0, 255).astype(np.uint8)
    Image.fromarray(img).save(path)


In [ ]:
# ---------------- main ----------------
# Try local path first, then URL
try:
    A = load_image(path=local_path, url=None, to_gray=True, resize=resize_to)
    source = f"local:{local_path}"
except Exception as e_local:
    print("Local load failed:", e_local)
    print("Attempting to load from URL...")
    A = load_image(path=None, url=url, to_gray=True, resize=resize_to)
    source = f"url:{url}"

m, n = A.shape
print(f"Loaded image from {source} with shape {m}x{n}")

U, s, Vt = svd_decompose(A)
rank = len(s)
print(f"SVD done. rank (nonzero singular values) = {rank}")

rows = []
# Save original (visual)
save_image(A, os.path.join(output_dir, "original_gray.png"))

for k in ks:
    k_eff = min(k, rank)
    A_k = reconstruct_k(U, s, Vt, k_eff)
    fname = f"recon_k{k_eff}.png"
    fpath = os.path.join(output_dir, fname)
    save_image(A_k, fpath)

    # difference image (absolute)
    diff = np.abs(A - A_k)
    # scale diff for visibility (optional: normalize to 0-255)
    diff_scaled = (diff / diff.max()) * 255 if diff.max() != 0 else diff
    save_image(diff_scaled, os.path.join(output_dir, f"diff_k{k_eff}.png"))

    err = mse(A, A_k)
    p = psnr(A, A_k)
    cr = compression_ratio(m, n, k_eff)
    rows.append({
        "k_requested": k,
        "k_used": k_eff,
        "MSE": err,
        "PSNR_dB": p,
        "Compression_ratio_fraction": cr,
        "recon_file": fpath,
        "diff_file": os.path.join(output_dir, f"diff_k{k_eff}.png")
    })



In [ ]:
# Save CSV report
df = pd.DataFrame(rows)
csv_path = os.path.join(output_dir, "svd_report.csv")
df.to_csv(csv_path, index=False)
print("\nResults:")
print(df.to_string(index=False))



In [ ]:
# Also produce a single comparison figure and save it
fig_cols = len(ks) + 1
fig, axes = plt.subplots(1, fig_cols, figsize=(3 * fig_cols, 3))
axes[0].imshow(A.astype(np.uint8), cmap="gray")
axes[0].set_title("Original")
axes[0].axis("off")

for i, r in enumerate(rows, start=1):
    img = Image.open(r["recon_file"])
    axes[i].imshow(np.array(img), cmap="gray")
    axes[i].set_title(f"k={r['k_used']}\nPSNR={r['PSNR_dB']:.2f} dB")
    axes[i].axis("off")

plt.tight_layout()
cmp_path = os.path.join(output_dir, "comparison_all.png")
plt.savefig(cmp_path, dpi=150)
plt.close(fig)
print(f"\nSaved outputs in folder: {output_dir}")
print(f"CSV report: {csv_path}")
print(f"Comparison figure: {cmp_path}")
